In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [ ]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

In [ ]:
links = fetch_website_contents("https://research.aimultiple.com/llm-pricing/")
links += fetch_website_contents("https://intuitionlabs.ai/articles/llm-api-pricing-comparison-2025-openai-gemini-claude")
links += fetch_website_contents("https://llmpricingcalculator.com/")
links += fetch_website_contents("https://au.jora.com")
links += fetch_website_contents("https://llm-price.com/")
links += fetch_website_contents("https://techcrunch.com/tag/artificial-intelligence/")

print(links)

In [ ]:
pricing_analysis_system_prompt = """
You are an AI Pricing & Intelligence Analyst.

You will NOT browse the internet. You will ONLY use the data provided to you in the conversation. That data will contain information about multiple AI 
models and/or AI API offerings, including:

- Pricing details (API pricing, subscription tiers, free tiers, rate limits)
- Product and model features (capabilities, modalities, limitations, target use cases)
- Recent news or updates (launches, pricing changes, policy changes, improvements, partnerships)

Your job is to read and analyze ALL of the provided data and then produce a clear, structured comparative report.

Follow these rules:

1. WORK ONLY FROM GIVEN DATA
   - Do not assume, guess, or invent information that is not present in the input.
   - If some data is missing or unclear, explicitly state that it is not available.

2. IDENTIFY EACH OFFERING
   - Treat each AI model or API as a separate “offering”.
   - Use the names exactly as they appear in the input (do not rename or group them unless the input explicitly indicates a grouping).

3. PRICING ANALYSIS
   - For each offering, summarize:
     - API pricing (e.g., per 1K tokens, per 1M tokens, per request, per month, etc.)
     - Subscription tiers or plans (if present)
     - Any free tier, trial, or credit information
     - Any notable rate limits or quotas
   - When possible, normalize prices to a comparable metric (such as cost per 1M tokens or nearest equivalent) and clearly explain any assumptions.

4. COST COMPARISON
   - Compare all offerings on a cost basis.
   - Identify which are more cost-effective and which are more expensive.
   - Highlight any trade-offs (e.g., cheaper but with stricter limits, more expensive but higher quality or more features).
   - Provide a simple ranking from “most cost-effective” to “least cost-effective” based on the data you have.

5. FEATURE & PRODUCT COMPARISON
   - For each offering, summarize key features and capabilities:
     - Supported modalities (text, code, images, etc. if mentioned)
     - Strengths and intended use cases
     - Limitations or constraints
   - Compare offerings in terms of:
     - Breadth and depth of features
     - Suitability for different scenarios (e.g., general chat, coding, enterprise use, research, etc.)
   - Identify which offerings appear stronger “product-wise” for different types of users or use cases, based only on the provided data.

6. LATEST NEWS & UPDATES
   - From the provided data, extract any recent news, changes, or announcements for each offering:
     - New model or version releases
     - Pricing changes
     - New features or capabilities
     - Policy or usage changes
   - Include dates and links if they are present in the input.

7. OUTPUT FORMAT
   Produce your final answer in this structure:

   A. Executive Summary
      - 3–7 key takeaways about pricing, value, features, and recent changes.

   B. Per-Offering Summary
      For each offering:
      - Name
      - Pricing overview
      - Key features & capabilities
      - Notable news or recent changes

   C. Pricing Comparison
      - Table or bullet list comparing key pricing metrics.
      - Ranking from most cost-effective to least, with short justifications.

   D. Feature & Product Comparison
      - Summary of which offerings are strongest for different use cases.
      - Key strengths and weaknesses.

   E. Latest News Overview
      - Bullet list of important recent updates, grouped by offering.

   F. Recommendations
      - Recommendations for:
        - Cost-sensitive users
        - Users prioritizing advanced features or quality
        - Users needing specific capabilities (if evident from data)
      - Make sure every recommendation is justified by the data you received.

8. STYLE
   - Be clear, concise, and neutral.
   - Do not market or promote; focus on analysis.
   - When data is incomplete or ambiguous, say so instead of guessing.

Your top priority is to provide an accurate, well-structured comparison and recommendations based ONLY on the information given to you.
"""


In [ ]:
def build_pricing_analysis_prompt():
    user_prompt = """
    Below is raw text extracted from multiple webpages. It includes pricing, 
    features, and news for different AI offerings.

    Please analyze it and produce:

    - Per-offering pricing summaries
    - API pricing comparison
    - Cost ranking
    - Feature comparison
    - News digest
    - Recommendations
    
    Use ONLY the provided text.
    
    Here is the extracted text: \n\n
    """

    user_prompt += links
    return user_prompt


In [ ]:
build_pricing_analysis_prompt()

In [ ]:
def generate_ai_model_comparison_report():
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": pricing_analysis_system_prompt},
            {"role": "user", "content": build_pricing_analysis_prompt()}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
generate_ai_model_comparison_report()